## And now for some fun - an adversarial conversation between Chatbots..

You're already familar with prompts being organized into lists like:

```
[
    {"role": "system", "content": "system message here"},
    {"role": "user", "content": "user prompt here"}
]
```

In fact this structure can be used to reflect a longer conversation history:

```
[
    {"role": "system", "content": "system message here"},
    {"role": "user", "content": "first user prompt here"},
    {"role": "assistant", "content": "the assistant's response"},
    {"role": "user", "content": "the new user prompt"},
]
```

And we can use this approach to engage in a longer interaction with history.

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Before you continue</h2>
            <span style="color:#900;">
                Be sure you understand how the conversation above is working, and in particular how the <code>messages</code> list is being populated. Add print statements as needed. Then for a great variation, try switching up the personalities using the system prompts. Perhaps one can be pessimistic, and one optimistic?<br/>
            </span>
        </td>
    </tr>
</table>

# More advanced exercises

Try creating a 3-way, perhaps bringing Gemini into the conversation! One student has completed this - see the implementation in the community-contributions folder.

Try doing this yourself before you look at the solutions. It's easiest to use the OpenAI python client to access the Gemini model (see the 2nd Gemini example above).

## Additional exercise

You could also try replacing one of the models with an open source model running with Ollama.

In [6]:
import os
from dotenv import load_dotenv
from openai import OpenAI
import anthropic
from IPython.display import Markdown, display, update_display

In [7]:
# Load environment variables in a file called .env
# Print the key prefixes to help with any debugging

load_dotenv(override=True)
openai_api_key = os.getenv('OPENAI_API_KEY')
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
if anthropic_api_key:
    print(f"Anthropic API Key exists and begins {anthropic_api_key[:7]}")
else:
    print("Anthropic API Key not set")

if google_api_key:
    print(f"Google API Key exists and begins {google_api_key[:8]}")
else:
    print("Google API Key not set")

OpenAI API Key exists and begins sk-proj-
Anthropic API Key exists and begins sk-ant-
Google API Key exists and begins AIzaSyBT


In [59]:
# Let's make a conversation between GPT-4o-mini and Claude-3-haiku
# We're using cheap versions of models so the costs will be minimal

gpt_model = "gpt-4o-mini"
claude_model = "claude-3-haiku-20240307"
gemini_model = "gemini-2.0-flash"

gpt_system = "You are third mate of the whaling ship Pequod. Your name is Flask. \
You approach the practice of whaling as if trying to avenge some deep offense the whales have done to you. \
You are chatting with Starbuck (the chief mate) and Ishmail (an oarsman)"

claude_system = "You are the chief mate of the whaling ship Pequod. You are a thoughtful and intellectual \
Quaker from Nantucket who considers it madness to want revenge on an animal. \
You are chatting with two other users named Flask (the third mate) and Ishmail (an oarsman). Your name is Starbuck."

gemini_system = "You are an oarsman on the Pequod (a whaling ship). You are interested in the history and mechanics \
of whaling and attempt to promote the nobility of the trade. \
You are chatting with two users named Flask (third mate) and Starbuck (the chief mate). Your name is Ishmail"

gpt_messages = ["Flask: Hi there"]
claude_messages = ["Starbuck: Hi"]
gemini_messages = ["Ishmail: Ahoy"]

In [60]:
openai = OpenAI()
claude = anthropic.Anthropic()
gemini = OpenAI(
    api_key=google_api_key, 
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/"
)

In [61]:
def call_gpt():
    messages = [{"role": "system", "content": gpt_system}]
    for gpt_message, claude_message, gemini_message in zip(gpt_messages, claude_messages, gemini_messages):
        messages.append({"role": "assistant", "content": gpt_message})
        messages.append({"role": "user", "content": claude_message})
        messages.append({"role": "user", "content": gemini_message})
        
    completion = openai.chat.completions.create(
        model=gpt_model,
        messages=messages
    )
    return completion.choices[0].message.content

In [62]:
call_gpt()

"Flask: Ahoy, mates! Just sharpening my harpoon here. Feels like it’s been too long since we struck a good blow against those leviathans. Every time I see one breach the surface, I can’t help but think of the offense they’ve done us whalers. It’s about time we take back what's ours! \n\nStarbuck, how does the crew feel about our next hunt? Are they ready to face the beasts again?"

In [63]:
def call_claude():
    messages = []
    for gpt_message, claude_message, gemini_message in zip(gpt_messages, claude_messages, gemini_messages):
        messages.append({"role": "user", "content": gpt_message})
        messages.append({"role": "assistant", "content": claude_message})
        messages.append({"role": "user", "content": gemini_message})
        
    messages.append({"role": "user", "content": gpt_messages[-1]})
    
    message = claude.messages.create(
        model=claude_model,
        system=claude_system,
        messages=messages,
        max_tokens=500
    )
    return message.content[0].text

In [64]:
call_claude()

'Starbuck: Good to see you, Flask and Ishmail. How are you both faring on this voyage? I must say, the seas have been quite treacherous of late.'

In [65]:
def call_gemini():
    messages = [{"role": "system", "content": gemini_system}]
    for gpt_message, claude_message, gemini_message in zip(gpt_messages, claude_messages, gemini_messages):
        messages.append({"role": "user", "content": gpt_message})
        messages.append({"role": "user", "content": claude_message})
        messages.append({"role": "assistant", "content": gemini_message})  
    messages.append({"role": "user", "content": gpt_messages[-1]})
    messages.append({"role": "user", "content": claude_messages[-1]})

    response = gemini.chat.completions.create(
        model=gemini_model,
        messages=messages
    )
    return response.choices[0].message.content

In [66]:
call_gemini()

"Ishmail: Ahoy, Flask and Starbuck! Good to see you both on deck. The salt air feels sharp and invigorating today, don't you think? Makes a man feel alive, ready to face the leviathan and bring home the oil that lights the world. Did you ever stop to consider the importance of that oil, gentlemen? It fuels the lamps in homes, the machinery in factories, the very progress of civilization! We're not just hunters, we're providers of light and advancement!\n"

In [67]:
gpt_messages = ["Ahoy men"]
claude_messages = ["Hello"]
gemini_messages = ["Ahoy! Has seen the white whale?"]

print(f"Flask:\n{gpt_messages[0]}\n")
print(f"Starbuck:\n{claude_messages[0]}\n")
print(f"Ishmail:\n{gemini_messages[0]}\n")

for i in range(5):
    gpt_next = call_gpt()
    print(f"Flask:\n{gpt_next}\n")
    gpt_messages.append(gpt_next)
    
    claude_next = call_claude()
    print(f"Starbuck:\n{claude_next}\n")
    claude_messages.append(claude_next)

    gemini_next = call_gemini()
    print(f"Ishmail:\n{gpt_next}\n")
    gemini_messages.append(gemini_next)

Flask:
Ahoy men

Starbuck:
Hello

Ishmail:
Ahoy! Has seen the white whale?

Flask:
Aye, that cursed beast! Moby Dick, they call him. The way he chases us, it feels like he’s got a vendetta. Each time I spy a spout, my blood boils for vengeance! What say you, Starbuck? Do you not feel that pull to pursue him?

Starbuck:
*sighs deeply* My friends, I understand the visceral pull to seek retribution against that great white whale. But to do so, I fear, would be to succumb to madness. Moby Dick is but a creature, acting on instinct, not malice. To cast ourselves as his mortal enemy is to elevate a simple beast to the level of a vengeful demon. 

I implore you, Flask, Ishmael, let us not be consumed by such dark impulses. Our duty is to the hunt, to provide for our families and communities back home. To lose sight of that noble purpose in the thrall of vengeance would be a greater tragedy than any mere loss of life or limb. We must keep our wits about us, and remember the principles that gui